In [10]:
!pip3 install --user pandas

     |████████████████████████████████| 10.4MB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 15.3MB 1.0MB/s eta 0:00:01     |████████████████████████████    | 13.4MB 1.3MB/s eta 0:00:02
     |████████████████████████████████| 512kB 1.0MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
from elasticsearch import helpers 
from collections.abc import MutableMapping
import pandas as pd
import numpy as np

def flatten(nested_dict, parent_key=''):
    items = []
    for k, v in nested_dict.items():
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, k).items())
        else:
            items.append((k, v))
    return dict(items)
    
    
query = {
  "sort": [
    {
      "timestamp": "desc"
    }
  ],
  "query": {
    "match_phrase": {
      "event": "Search result selected"
    }
  },
  "size": 10000
}

response = es.search(
    body=query,
    index="search_relevance_implicit"
)

df = pd.DataFrame([
    flatten(event['_source']) for event in response['hits']['hits']
])

In [12]:
#note: Python client automatically indexes from latest to earliest.  To check start date:

sorted=df.sort_values(by=['timestamp'], ascending=True) 
sorted.head()

,event,anonymousId,timestamp,network,toggles,aggregations,items.locations.accessConditions.status,items.locations.locationType,page,production.dates.from,...,sortOrder,workType,id,position,resultIdentifiers,resultSubjects,resultWorkType,source,resultLanguage,_queryType
9999,Search result selected,09a2b225-4442-4868-bb74-45c7ec3c6a0b,2020-09-15T02:27:11.494Z,None,[availableOnline:true],[workType],"[!open-with-advisory, !restricted, !closed]","[iiif-image, iiif-presentation]",1,None,...,None,"[a, b, g, i, k, l, q]",yj52w5vt,0,"[b30496767, 3049676]",[Schizophrenia - Pictorial works - Personal na...,Pictures,work_result,English,NaN
9998,Search result selected,09a2b225-4442-4868-bb74-45c7ec3c6a0b,2020-09-15T02:27:32.877Z,None,[availableOnline:true],[workType],"[!open-with-advisory, !restricted, !closed]","[iiif-image, iiif-presentation]",1,None,...,None,"[a, b, g, i, k, l, q]",exwd7bm3,0,"[L0038204, GC WR 17 1899M93a3E]","[Venereal Diseases, Hands, Fingernail, Sexuall...",Digital Images,work_result,NaN,NaN
9997,Search result selected,09a2b225-4442-4868-bb74-45c7ec3c6a0b,2020-09-15T02:27:51.006Z,None,[availableOnline:true],[workType],"[!open-with-advisory, !restricted, !closed]","[iiif-image, iiif-presentation]",1,None,...,None,"[a, b, g, i, k, l, q]",jzhummq3,0,[B0011047],"[Neurodegeneration, Forgetfulness, Old Age, Ar...",Digital Images,work_result,NaN,NaN
9996,Search result selected,200e82ab-b679-4e44-8357-49f310e48b4c,2020-09-15T02:27:56.242Z,None,[],[workType],"[!open-with-advisory, !restricted, !closed]","[iiif-image, iiif-presentation]",1,None,...,None,"[a, b, g, i, k, l, q]",x7dcqwz7,5,"[L0032706, EPB RAMC ROL VOL 1, EPB RAMC ROL VOL1]","[EPB RAMC ROL VOL 1, ROLLO, JOHN D. 1809, EPB ...",Digital Images,work_result,NaN,NaN
9995,Search result selected,200e82ab-b679-4e44-8357-49f310e48b4c,2020-09-15T02:28:33.344Z,None,[],[workType],"[!open-with-advisory, !restricted, !closed]","[iiif-image, iiif-presentation]",1,None,...,None,"[a, b, g, i, k, l, q]",x7dcqwz7,5,"[L0032706, EPB RAMC ROL VOL 1, EPB RAMC ROL VOL1]","[EPB RAMC ROL VOL 1, ROLLO, JOHN D. 1809, EPB ...",Digital Images,work_result,NaN,NaN


In [ ]:
#data cleansing

df=df.loc[df['network'] != 'StaffCorporateDevices'] #remove staff usage
#df=df.loc[df['timestamp'] > '2019-11-07 00:00:00']  #only use searches after AND implemented
#df=df.loc[df['event'] != 'Search landing']

df2=df.loc[df['timestamp'] >= '2020-03-03 00:00:00']  #grabs 2 days' of data, ie from 1/3/20


df2.head(5)

In [ ]:
df.sort_values(by=['anonymousId','timestamp'])
unique_selects=df.drop_duplicates(subset='anonymousId', keep='first')
unique_selects.head(5)

In [ ]:
#How many workIds?
summary=unique_selects.groupby('id').count()[['anonymousId']]
summary


In [ ]:
#How many workIds viewed once?
viewed_once=summary.loc[summary['anonymousId']<=1]
count=viewed_once['anonymousId'].count()
print(count)

In [ ]:
#How many workIds viewed twice?
viewed_twice=summary.loc[summary['anonymousId']==2]
count=viewed_twice['anonymousId'].count()
print(count)

In [ ]:
#Create bar chart for workIds viewed more than twice.

summary.sort_values(by='anonymousId', ascending=False)[:19].plot.bar(legend=False)

In [ ]:
#calculate skewness

skew=summary.skew()
print(skew)

In [ ]:
#output data to csv to check

summary.to_csv('skew_for_2_days.csv') 